# Work in the background

#### Installs & Imports

In [1]:
!pip install transformers==2.8.0

     |████████████████████████████████| 573kB 7.7MB/s 
     |████████████████████████████████| 890kB 26.6MB/s 
     |████████████████████████████████| 1.1MB 19.9MB/s 
     |████████████████████████████████| 133kB 51.8MB/s 
     |████████████████████████████████| 3.7MB 47.1MB/s 
     |████████████████████████████████| 7.2MB 53.1MB/s 
     |████████████████████████████████| 71kB 11.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=110c1d25ee994f096766d96a7f0b49640337a70f6d911cd15baaa5783a52a72a
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
ERROR: botocore 1.19.47 has requirement urllib3<1.27,>=1.25.4; python_version != "3.4", but you'll have urllib3 1.24.3 which is incompatible.


In [2]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizerFast
from transformers import BertModel
import torch.nn as nn
import transformers
import numpy as np
import warnings
import torch
import re

In [3]:
device = torch.device("cuda")
print(f'There are {torch.cuda.device_count()} GPU(s) available.')
print('Device name:', torch.cuda.get_device_name(0))

There are 1 GPU(s) available.
Device name: Tesla T4


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##### test

**From the New Yorker 2-01-2021**

When news networks called Georgia for Joe Biden, on November 13th, Nsé Ufot, the C.E.O. of the New Georgia Project, was atop Stone Mountain, a hunk of granite east of Atlanta that is home to the largest Confederate monument on earth. She was with colleagues, taking a staff photo and celebrating the work they’d done to turn out voters in record numbers. As word of Biden’s victory spread, some people teared up, Ufot said, but not her. Her organization, which was founded, in 2014, by Stacey Abrams, had made three million phone calls, sent two million text messages, and carefully knocked on half a million doors. Biden won the state by around twelve thousand votes. The New Georgia Project’s efforts, and those of a handful of similar groups, put him over the top. They also insured that Georgia would have not one but two runoff elections for the U.S. Senate, on January 5th, pitting a pair of Republican incumbents against Democratic challengers. I asked Ufot whether Georgia had “turned blue,” as headlines proclaimed. “I get the desire to, like, make scorching takes,” she said. “But can we just enjoy this before we get back to work?”

**From Before it's news (a known fake news website) 3-01-2021**

Vice President Michael Pence is preparing to flee the United States to a non-extradition country in response to credible evidence that his boss, President Donald J. Trump, intends to charge him with treasonous crimes against America and its people.
On 1 January Lin Wood, a conservative attorney and Trump Ally who has fought many failed battles to overturn the result of the 2020 election, told Trump that he had found information linking the VP to the Deep State. Wood said Pence was a “trojan horse” in the White House who had been aiding and abetting “agents of evil” for almost four years. The controversial attorney, whom the mainstream media has called a conspiratorial retard, accused Pence of clandestinely meeting with Biden and Adam Schiff several times during the last two weeks of September.
The evening after giving Trump evidence of Pence’s betrayal, Wood went on a tweetstorm calling on the president to label Pence an insurrectionist and have him executed via a firing squad.

#### Bert e preprocessing

In [9]:
class BertTuned(nn.Module):

    def __init__(self):
        super(BertTuned, self).__init__()
        self.bert = transformers.BertModel.from_pretrained('bert-base-uncased')
        self.bert_drop = nn.Dropout(0.35)
        self.fc = nn.Linear(768, 1)

    def forward(self, ids, mask):
        _, out1 = self.bert(ids, attention_mask=mask)
        out1 = self.bert_drop(out1)
        output = self.fc(out1)
        return output

In [ ]:
model = BertTuned()
model.load_state_dict(torch.load('drive/MyDrive/modelF5.pt'))
model.to(device)

In [13]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

In [14]:
def detectFakeNews():

  print("Please insert part of the article you would like to examin")
  article = input()
  encoded_dict = tokenizer.encode_plus( text = article, add_special_tokens = True, max_length = 512,           
                                      pad_to_max_length=True, truncation = True, return_attention_mask = True)
  
  ids = [encoded_dict['input_ids']]
  mask = [encoded_dict['attention_mask']]
  ids_ = torch.LongTensor(ids).to(device)
  mask_ = torch.LongTensor(mask).to(device)
  model.eval()
  with torch.no_grad():
    preds = model(ids_, mask_)
  output = torch.round(torch.sigmoid(preds).flatten()).cpu().detach().numpy().tolist()
  if output[0] == 0.0:
    print("This article is reliable")
  elif output[0] == 1.0:
    print("This article is not reliable")
  else:
    print("Something went wrong. Please insert simple text")

# Let's try it out

In [ ]:
detectFakeNews()

**From the New York Times**

Thieves Take a Chunk of Change, All 221 Pounds of It, From a Berlin Museum - Melissa Eddy


BERLIN  —   You could never palm it, flip it or plunk it into a vending machine. But apparently it can be pinched: One of the world’s largest gold coins, a   Canadian monster called the Big Maple Leaf, was stolen overnight from the Bode Museum in Berlin, the police said on Monday. The coin is about 21 inches in diameter and over an inch thick. It has the head of Queen Elizabeth II on one side and a maple leaf on the other. Its face value is 1 million Canadian dollars, or about \$750, 000, but by gold content alone, it is worth as much as \$4. 5 million at current market prices. And though it weighs about as much as a refrigerator, somehow thieves apparently managed to lug it through the museum and up at least one floor to get it out of a window at the back of the building. The police are still trying to figure out exactly how they did it. The Bode Museum, which sits on Museum Island in the Spree River, is part of the complex belonging to the Prussian Cultural Heritage Foundation, or in German, the Preussischer Kulturbesitz. The local   commuter railway runs across the island along the back of the museum. The burglars seemed to have broken in through a window above the railway tracks during the     hours when the trains pause for the night. The police were alerted to the   at 4 a. m. and think that it took place between 3:20 a. m. and 3:45 a. m.  [....]

**From RonPaulLibertyReport.com**

The Rise of Mandatory Vaccinations Means the End of Medical Freedom - Shaun Bradley

Mandatory vaccinations are about to open up a new frontier for government control. Through the war on drugs, bureaucrats arbitrarily dictate what people can and can’t put into their bodies, but that violation pales in comparison to forcibly medicating millions against their will. Voluntary and informed consent are essential in securing individual rights, and without it, self-ownership will never be respected.

The liberal stronghold of California is trailblazing the encroaching new practice and recently passed laws mandating that children and adults must have certain immunizations before being able to attend schools or work in certain professions. The longstanding religious and philosophical exemptions that protect freedom of choice have been systematically crushed by the state. [...]